## Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Indian restaurant** in **Dubai**, UAE.

Since there are lots of restaurants in Berlin we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Indian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Dubai center will be obtained using **Google Maps API geocoding** of well known Dubai location (Dubai Mall)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Dubai Mall.

Let's first find the latitude & longitude of Dubai Mall, using specific, well known address and Google Maps geocoding API.

In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [11]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [13]:
import requests

address = 'Dubai Mall, UAE'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.1970924 55.2790356376064


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Dubai Mall. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).


In [20]:
search_query = 'Indian'
radius = 10000
print(search_query + ' .... OK!')

Indian .... OK!


In [21]:
CLIENT_ID = 'F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ' # your Foursquare ID
CLIENT_SECRET = 'QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ
CLIENT_SECRET:QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152


In [22]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=F0YFADQU0DZIMY2JGB3JZH3W50XKPWGJY0NDPNSTVUUIMYVQ&client_secret=QZFK4RNSPGS3BQUW3BIAFLWHG1WJ4QPCSRXFT4HJVTDAE152&ll=25.1970924,55.2790356376064&v=20180604&query=Indian&radius=10000&limit=30'

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0f7453a6ec980039c463ba'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '4bcab216cc8cd13a0c41becf',
    'location': {'address': 'Wafi, Oud Metha',
     'cc': 'AE',
     'city': 'دبي',
     'country': 'الإمارات العربية المتحدة',
     'crossStreet': 'Between Sheikh Rashid Road and 26th Street',
     'distance': 5467,
     'formattedAddress': ['Wafi, Oud Metha (Between Sheikh Rashid Road and 26th Street)',
      'دبي',
      'الإمارات العربية المتحدة'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.229791335703393,
       'lng': 55.31953950687779}],
     'lat': 25.229791335703393,
     'lng': 55.31953950687779,
     'postalCode': '30567',
  

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'shortName': 'Indian', 'pluralName': 'Indian...",False,4bcab216cc8cd13a0c41becf,"Wafi, Oud Metha",AE,دبي,الإمارات العربية المتحدة,Between Sheikh Rashid Road and 26th Street,5467,"[Wafi, Oud Metha (Between Sheikh Rashid Road a...","[{'label': 'display', 'lat': 25.22979133570339...",25.229791,55.319540,NaN,30567,دبي,Asha's - Contemporary Indian Restaurant,v-1561293908
1,"[{'shortName': 'Indian', 'pluralName': 'Indian...",False,5be56ba5646e38002c6fb889,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'label': 'display', 'lat': 25.18795, 'lng': ...",25.187950,55.297947,Za'abeel,NaN,دبي,Mohalla Indian Street Food,v-1561293908
2,"[{'shortName': 'Indian', 'pluralName': 'Indian...",False,4b9a6e19f964a5208cb435e3,Dubai Festival City,AE,دبي,الإمارات العربية المتحدة,NaN,7775,"[Dubai Festival City, دبي, الإمارات العربية ال...","[{'label': 'display', 'lat': 25.22120819857269...",25.221208,55.351495,NaN,NaN,دبي,Indian Palace Restaurant (مطعم القصر الهندي) (...,v-1561293908
3,"[{'shortName': 'Indian', 'pluralName': 'Indian...",False,4b99106ff964a520fb5e35e3,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 25.19672003392247...",25.196720,55.276636,NaN,NaN,دبي,India Palace Express,v-1561293908
4,"[{'shortName': 'Indian', 'pluralName': 'Indian...",False,4d492c26f53c8cfae38a1447,Mall of Emirates,AE,NaN,الإمارات العربية المتحدة,NaN,11920,"[Mall of Emirates, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 25.11850169419072...",25.118502,55.198675,NaN,NaN,NaN,Indian Pavilion,v-1561293908


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Asha's - Contemporary Indian Restaurant,Indian Restaurant,"Wafi, Oud Metha",AE,دبي,الإمارات العربية المتحدة,Between Sheikh Rashid Road and 26th Street,5467,"[Wafi, Oud Metha (Between Sheikh Rashid Road a...","[{'label': 'display', 'lat': 25.22979133570339...",25.229791,55.319540,NaN,30567,دبي,4bcab216cc8cd13a0c41becf
1,Mohalla Indian Street Food,Indian Restaurant,Dubai Design District D3,AE,دبي,الإمارات العربية المتحدة,NaN,2159,"[Dubai Design District D3, دبي, الإمارات العرب...","[{'label': 'display', 'lat': 25.18795, 'lng': ...",25.187950,55.297947,Za'abeel,NaN,دبي,5be56ba5646e38002c6fb889
2,Indian Palace Restaurant (مطعم القصر الهندي) (...,Indian Restaurant,Dubai Festival City,AE,دبي,الإمارات العربية المتحدة,NaN,7775,"[Dubai Festival City, دبي, الإمارات العربية ال...","[{'label': 'display', 'lat': 25.22120819857269...",25.221208,55.351495,NaN,NaN,دبي,4b9a6e19f964a5208cb435e3
3,India Palace Express,Indian Restaurant,The Dubai Mall,AE,دبي,الإمارات العربية المتحدة,NaN,245,"[The Dubai Mall, دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 25.19672003392247...",25.196720,55.276636,NaN,NaN,دبي,4b99106ff964a520fb5e35e3
4,Indian Pavilion,Indian Restaurant,Mall of Emirates,AE,NaN,الإمارات العربية المتحدة,NaN,11920,"[Mall of Emirates, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 25.11850169419072...",25.118502,55.198675,NaN,NaN,NaN,4d492c26f53c8cfae38a1447
5,Metro Grand Indian Restaurant,Asian Restaurant,NaN,AE,دبي,الإمارات العربية المتحدة,NaN,1660,"[دبي, الإمارات العربية المتحدة]","[{'label': 'display', 'lat': 25.21037743663270...",25.210377,55.271535,NaN,NaN,دبي,548130c6498e82fa71d319e4
6,Lazeez Indian Cuisine,Indian Restaurant,Wasl Square,AE,دبي,الإمارات العربية المتحدة,Safa Park Road,4022,"[Wasl Square (Safa Park Road), دبي, الإمارات ا...","[{'label': 'display', 'lat': 25.185643, 'lng':...",25.185643,55.241163,NaN,NaN,دبي,5a0f13df1755622302f4fe6d
7,Indian Handy Craft,Arts & Crafts Store,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,3103,[الإمارات العربية المتحدة],"[{'label': 'display', 'lat': 25.176758, 'lng':...",25.176758,55.257954,NaN,NaN,NaN,5016c225e4b0ad6ab3c135f0
8,Gazebo Indian cuisine DIFC,Indian Restaurant,NaN,AE,NaN,الإمارات العربية المتحدة,NaN,2290,[الإمارات العربية المتحدة],"[{'label': 'display', 'lat': 25.217665, 'lng':...",25.217665,55.278635,NaN,NaN,NaN,56df0c60498e77585c2fb553
9,Indian Summer,Indian Restaurant,Umm Squeim01 Jumeirah Beach Road Oppt Enoc,AE,دبي,الإمارات العربية المتحدة,NaN,7005,"[Umm Squeim01 Jumeirah Beach Road Oppt Enoc, د...","[{'label': 'display', 'lat': 25.169991, 'lng':...",25.169991,55.216270,NaN,NaN,دبي,591048aff00a703cb8d95695
